# 소프트맥스 회귀 
Softmax Regression
:3개 이상의 선택지로부터 1개를 선택하는 문제인 다중 클래스 분류를 풀기 위한 방법

# One-Hot Encoding
- 범주형 데이터를 처리할 때 레이블을 표현하는 방법
- 선택해야 하는 선택지의 개수만큼 차원을 가지면서, 각 선택지의 인덱스에 해당하는 원소에는 1, 나머지 원소에는 0의 값을 할당하는 표현 방법
- 원-핫 인코딩으로 표현된 벡터를 원-핫 벡터라고 함
- 대부분의 다중 클래스 분류 문제가 각 클래스 간의 관계가 균등함 -> 원-핫 벡터는 이를 잘 표현할 수 있음

1. 레이블링 방법
- 직관적 - 분류해야 할 클래스 전체에 정수 인코딩을 하는 것
    - 손실 함수 계산 시 오차가 더 큼 (범주형 변수 간의 레이블링은 각 차이가 의미가 없는데, 단순히 숫자로만 접근하면 의미가 있는 것처럼 보이기 때문)
    - Apple 1, banana 2, tomato 3일 때 tomato랑 apple이 먼 것이 아님!
    - 만약 각 클래스가 순서의 의미를 갖고 있어 회귀를 통해 분류 문제를 풀 수 있을 때는 좋음
- 다중 클래스 분류 문제에서는 원-핫 인코딩을 사용하는 것이 더 좋음 (클래스의 성질을 잘 표현했다고 할 수 있음)
    - 모든 쌍에 대해 유클리드 거리를 구해도 모두 유클리드 거리가 동일
    - 각 클래스의 표현 방법이 무작위성을 가진다는 점을 표현할 수 있음!

# 소프트맥스 회귀 이해하기

1. 다중 클래스 분류
: 세 개 이상의 답 중 하나를 고르는 문제

- 로지스틱 회귀의 확률의 총 합이 1이 되는 이 아이디어를 다중 클래스 분류 문제에 적용함
- 각 클래스, 즉 각 선택지마다 소수 확률을 할당함 (총 합 1)

2. 소프트맥스 함수
: 분류해야하는 정답지(클래스)의 총 개수를 k라고 할 때, k차원의 벡터를 입력받아 각 클래스에 대한 확률을 추정함

<분꽃 데이터를 이용한 이해>
- 하나의 샘플 데이터는 4개의 독립 변수 x를 가짐
- 하지만 소프트맥스의 함수의 입력으로 사용되는 벡터는 벡터의 차원이 분류하고자 하는 클래스의 개수, 즉 3개가 되어야 함
- 차원 축소 방법: 소프트맥스 함수의 입력 벡터 z의 차원수만큼 결과값이 나오도록 가중치 곱을 진행 (4개의 독립변수가 하나의 차원으로 화살표를 쏨 -> 가중치)
- 예측값이 나왔다고 해보자. {0.26, 0.71, 0.04}
- 실제값은  {0, 1, 0}
- 현재 풀고 있는 샘플 데이터의 실제값이 setosa(두 번째)이면 예측값과 실제값이 오차가 되는 경우는 소프트맥스 함수의 결과가 [0 1 0]일 때
- 두 벡터의 오차를 계산하기 위해 소프트맥스 회귀는 비용 함수로 **크로스 엔트로피 함수** 사용

3. 비용 함수 - 크로스 엔프로피 함수


In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

In [2]:
z = torch.FloatTensor([1,2,3])
hypothesis = F.softmax(z, dim = 0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [3]:
hypothesis.sum()

tensor(1.)

In [4]:
z = torch.rand(3,5, requires_grad=True)

In [5]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [6]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [7]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [8]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [9]:
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [10]:
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [11]:
(y_one_hot*-F.log_softmax(z, dim =1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [12]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [13]:
F.cross_entropy(z, y)
# 비용 함수에 소프트맥스 함수까지 포함하고 있음을 기억!

tensor(1.4689, grad_fn=<NllLossBackward0>)

소프트 맥스 회귀 구현

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [15]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [17]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [18]:
#원-핫 인코딩을 한 후 y = (8*3)
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [19]:
#w 행렬의 크기 (4*3)

#모델 초기화
w = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

#optimizer 설정
optimizer = optim.SGD([w, b], lr=0.1)

In [20]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    #가설
    hypothesis = F.softmax(x_train.matmul(w) + b, dim=1)

    #비용 함수
    cost = (y_one_hot* -torch.log(hypothesis)).sum(dim=1).mean()

    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    #100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [22]:
#F.cross_entropy()는 그 자체로 소프트맥스 함수를 포함하고 있으므로 가설에서 소프트맥스 함수를 사용할 필요 x

w = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([w, b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    #cost 계산
    z = x_train.matmul(w)+b
    cost = F.cross_entropy(z, y_train)

    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [23]:
model = nn.Linear(4, 3)

#4개의 특성을 가지고 3개의 클래스로 분류

In [25]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    #H(x)계산
    prediction = model(x_train)
    #cost계산
    cost = F.cross_entropy(prediction, y_train)
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.254477
Epoch  100/1000 Cost: 0.234852
Epoch  200/1000 Cost: 0.223383
Epoch  300/1000 Cost: 0.212954
Epoch  400/1000 Cost: 0.203418
Epoch  500/1000 Cost: 0.194667
Epoch  600/1000 Cost: 0.186609
Epoch  700/1000 Cost: 0.179166
Epoch  800/1000 Cost: 0.172271
Epoch  900/1000 Cost: 0.165868
Epoch 1000/1000 Cost: 0.159906


In [26]:
class softmaxClassifierModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.linear = nn.Linear(4,3)
    def Forward(self, x):
        return self.linear(x)

In [27]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.159848
Epoch  100/1000 Cost: 0.154288
Epoch  200/1000 Cost: 0.149089
Epoch  300/1000 Cost: 0.144216
Epoch  400/1000 Cost: 0.139641
Epoch  500/1000 Cost: 0.135338
Epoch  600/1000 Cost: 0.131284
Epoch  700/1000 Cost: 0.127459
Epoch  800/1000 Cost: 0.123843
Epoch  900/1000 Cost: 0.120421
Epoch 1000/1000 Cost: 0.117177


# 소프트맥스 회귀로 MNIST 데이터 분류하기
## MNIST: 숫자 0부터 9까지의 이미지로 구성된 손글씨 데이터셋

- torchvision은 유명한 데이터셋들, 이미 구현되어져 있는 유명한 모델들, 일반적인 **이미지** 전처리 도구들을 포함하고 있는 패키지 (자연어 처리는 torchtext)

In [28]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

USE_CUVA = torch.cuda.is_available()
device = torch.device('cuda' if USE_CUVA else 'cpu')
print("다음 기기로 학습합니다: ", device)

다음 기기로 학습합니다:  cuda


In [30]:
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [31]:
training_epochs = 15
batch_size = 100

In [29]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

9913344it [00:01, 5090276.62it/s]                             


Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



29696it [00:00, 911842.60it/s]           


Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



1649664it [00:01, 1597241.42it/s]                             


Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



5120it [00:00, 5503546.00it/s]          

Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



In [32]:
# dataset loader
data_loader = DataLoader(dataset=mnist_train,
                                          batch_size=batch_size, # 배치 크기는 100
                                          shuffle=True,
                                          drop_last=True)

In [33]:
linear = nn.Linear(784, 10, bias=True).to(device)

In [34]:
criterion = nn.CrossEntropyLoss().to(device)
#내부적으로 소프트맥스 함수를 포함하고 있음
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.1)

In [36]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)

    for X, Y in data_loader:
        X = X.view(-1, 28*28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.534610033
Epoch: 0002 cost = 0.359223902
Epoch: 0003 cost = 0.331266344
Epoch: 0004 cost = 0.316493899
Epoch: 0005 cost = 0.307034910
Epoch: 0006 cost = 0.300361186
Epoch: 0007 cost = 0.295011610
Epoch: 0008 cost = 0.290796012
Epoch: 0009 cost = 0.287290961


In [ ]:
#테스트 데이터를 사용하여 모델을 테스트함
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # MNIST 테스트 데이터에서 무작위로 하나를 뽑아서 예측을 해본다
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

    plt.imshow(mnist_test.test_data[r:r + 1].view(28, 28), cmap='Greys', interpolation='nearest')
    plt.show()